# Base Model Evaluation


## Overview
Evaluation is a crucial step allowing us to measure the performance and generalization ability of our models. In this notebook, we will systematically evaluate the base model using appropriate metrics and validation datasets before fine tuning it on BMO data.

- **Base Model Evaluation:**  
    We will assess the initial performance of the base model to establish a benchmark. This helps us understand how well the model performs before any task-specific adaptation.


Throughout this notebook, we will use visualizations and quantitative metrics to provide a comprehensive analysis of model performance. This approach ensures transparency and helps guide further improvements.

## Install Dependencies

In [ ]:
!pip install -qqU .

## Setup Paths and Directories

In [ ]:
import os
from pathlib import Path

from dotenv import load_dotenv

load_dotenv()

WORKSPACE = Path.cwd().parent  # Path to the workspace directory

OUTPUT_DIR = WORKSPACE / "output"

MODEL_NAME = os.getenv("STUDENT_MODEL_NAME", "RedHatAI/Llama-3.1-8B-Instruct")

# Create the output directory if not available
OUTPUT_DIR.mkdir(parents=True, exist_ok=True)

MODEL_PATH = OUTPUT_DIR / "base_model" / MODEL_NAME.replace("/", "__")


print(f"Model name : {MODEL_NAME}")

## Save the Model

In [ ]:
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer

# Save the model locally, for easier access for following steps
if not MODEL_PATH.exists():
    print("Model not available locally, Downloading the model locally ")

    # Save the model
    print(f"Loading model {MODEL_NAME}")
    model = AutoModelForCausalLM.from_pretrained(MODEL_NAME)
    model.save_pretrained(MODEL_PATH)
    print(f"Model saved to {MODEL_PATH}")

    # Save the tokenizer
    print(f"Loading tokenizer {MODEL_NAME}")
    tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)
    tokenizer.save_pretrained(MODEL_PATH)
    print(f"Tokenizer saved to {MODEL_PATH}")

    del model
    del tokenizer
else:
    print(f"Model Available locally : {MODEL_PATH}")

## Load the Model

In [ ]:
model = AutoModelForCausalLM.from_pretrained(
    MODEL_PATH, dtype=torch.float16, device_map="cuda:0"
)
tokenizer = AutoTokenizer.from_pretrained(MODEL_PATH)

print("✅ Successfully Loaded the model")

## LLM Sampling Parameters

We will use the below when testing our model

In [ ]:
################################################################################
# 🎯 Sampling/Generation Parameters                                            #
################################################################################
MAX_NEW_TOKENS = 256
DO_SAMPLE = True
TEMPERATURE = 0.7  # Meta's recommended temperature for Llama
TOP_P = 0.9  # Standard top_p for Llama models

print(f"MAX_NEW_TOKENS: {MAX_NEW_TOKENS}")
print(f"DO_SAMPLE: {DO_SAMPLE}")
print(f"TEMPERATURE: {TEMPERATURE}")
print(f"TOP_P: {TOP_P}")
print("✅ LLM sampling parameters defined")
print()
print("📊 Using Meta's recommended Llama sampling settings:")
print("  • Temperature 0.6 for balanced creativity/consistency")
print("  • Top-p 0.9 for good token diversity")
print("  • Stop on both EOS and <|eot_id|> tokens")

## Utility Functions

In [ ]:
from rich import print as pprint


def prompt_runner(model, tokenizer, prompt):
    inputs = tokenizer(prompt, return_tensors="pt").to(model.device)

    with torch.no_grad():
        outputs = model.generate(
            **inputs,
            max_new_tokens=MAX_NEW_TOKENS,
            temperature=TEMPERATURE,
            do_sample=DO_SAMPLE,
            top_p=TOP_P,
        )

    response = tokenizer.decode(
        outputs[0][inputs.input_ids.shape[1] :], skip_special_tokens=True
    )

    return response


def run_experimentation(prompt):
    # Run prompt in base model
    base_model_response = prompt_runner(model=model, tokenizer=tokenizer, prompt=prompt)

    # Print the response of the model

    pprint(f"""
    [bold]EXPERIMENTATION DETAILS[/bold]:
        MODEL NAME     : {MODEL_NAME}
        MAX NEW TOKENS : {MAX_NEW_TOKENS}
        DO SAMPLE      : {DO_SAMPLE}
        TEMPERATURE    : {TEMPERATURE}
        TOP P          : {TOP_P}


    [bold]PROMPT 💬[/bold]:

        [green]{prompt}[/green]

    [bold]MODEL RESPONSE 🤖[/bold]:

        {base_model_response}

    """)

## Test 1

We will test the knowledge of the model on BMO data. 

Question: `what is the meaning of verifying the identity of a person or an entity`

In [ ]:
prompt = """what is the meaning of verifying the identity of a person or an entity"""


run_experimentation(prompt)